In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 8.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=a6ec1a30b23cccdc95c8e011bdb48d3e66f7f9e8c54f48cdad572c1795b9965b
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


## Loading data

In [3]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType
from pyspark.ml.recommendation import ALS, ALSModel
from pyspark.ml.evaluation import RegressionEvaluator

In [4]:
spark = SparkSession.builder.appName("recommender-system").getOrCreate()

In [5]:
Games = spark.read.csv("/content/drive/MyDrive/games.csv", header='true',inferSchema='true')
Ratings = spark.read.csv("/content/drive/MyDrive/ratings.csv", header='true',inferSchema='true')

In [6]:
Games.show()

+-------+--------------------+------------+--------------------+--------------------+
|game_id|                name|release_date|             summary|          meta_score|
+-------+--------------------+------------+--------------------+--------------------+
|      1|The Legend of Zel...|   23-Nov-98|As a young boy, L...|                  99|
|      2|Tony Hawk's Pro S...|   20-Sep-00|As most major pub...|                  98|
|      3| Grand Theft Auto IV|   29-Apr-08|"[Metacritic's 20...| fresh off the bo...|
|      4|         SoulCalibur|    8-Sep-99|This is a tale of...|                  98|
|      5|  Super Mario Galaxy|   12-Nov-07|[Metacritic's 200...|                  97|
|      6|Super Mario Galaxy 2|   23-May-10|Super Mario Galax...|                  97|
|      7|Red Dead Redempti...|   26-Oct-18|Developed by the ...|                  97|
|      8|  Grand Theft Auto V|   18-Nov-14|Grand Theft Auto ...|                  97|
|      9|Disco Elysium: Th...|   30-Mar-21|Disco Elysi

In [7]:
Ratings.show()

+-------+-------+------+
|game_id|user_id|rating|
+-------+-------+------+
|      1|    314|     5|
|      1|    439|     3|
|      1|    588|     5|
|      1|   1169|     4|
|      1|   1185|     4|
|      1|   2077|     4|
|      1|   2487|     4|
|      1|   2900|     5|
|      1|   3662|     4|
|      1|   3922|     5|
|      1|   5379|     5|
|      1|   5461|     3|
|      1|   5885|     5|
|      1|   6630|     5|
|      1|   7563|     3|
|      1|   9246|     1|
|      1|  10140|     4|
|      1|  10146|     5|
|      1|  10246|     4|
|      1|  10335|     4|
+-------+-------+------+
only showing top 20 rows



In [8]:
from pyspark.sql.functions import max

In [9]:
print(Ratings.select(max('rating')).show())

+-----------+
|max(rating)|
+-----------+
|          5|
+-----------+

None


<div align="right" dir="auto">
برای اینکه بتوانیم روی دیتای لود شده به وسیله تابع  spark.sql
    کوئری‌های SQL بزنیم
    باید دو دیتاست لود شده را به عنوان table
    به spark 
    معرفی کنیم
</div>

In [ ]:
# for applying sql commands in pyspark we should use 
Ratings.registerTempTable("Ratings")
Games.registerTempTable("Games")

<div align="right" dir="auto">
برای اینکه تمام اطلاعات را در کنار همدیگر مشاهده کنیم نیاز به جوین کردن داده‌ها داریم.

</div>

In [11]:
all_data = spark.sql("SELECT Games.game_id, Ratings.user_id, Ratings.rating, Games.name, Games.release_date, Games.summary, Games.meta_score FROM Ratings INNER JOIN Games On Ratings.game_id = Games.game_id")
all_data.show()

+-------+-------+------+--------------------+------------+--------------------+----------+
|game_id|user_id|rating|                name|release_date|             summary|meta_score|
+-------+-------+------+--------------------+------------+--------------------+----------+
|      1|    314|     5|The Legend of Zel...|   23-Nov-98|As a young boy, L...|        99|
|      1|    439|     3|The Legend of Zel...|   23-Nov-98|As a young boy, L...|        99|
|      1|    588|     5|The Legend of Zel...|   23-Nov-98|As a young boy, L...|        99|
|      1|   1169|     4|The Legend of Zel...|   23-Nov-98|As a young boy, L...|        99|
|      1|   1185|     4|The Legend of Zel...|   23-Nov-98|As a young boy, L...|        99|
|      1|   2077|     4|The Legend of Zel...|   23-Nov-98|As a young boy, L...|        99|
|      1|   2487|     4|The Legend of Zel...|   23-Nov-98|As a young boy, L...|        99|
|      1|   2900|     5|The Legend of Zel...|   23-Nov-98|As a young boy, L...|        99|

In [17]:
print(Ratings.filter(Ratings['user_id'] == 588).show())

+-------+-------+------+
|game_id|user_id|rating|
+-------+-------+------+
|      1|    588|     5|
|      3|    588|     1|
|      8|    588|     3|
|      9|    588|     3|
|     11|    588|     4|
|     15|    588|     4|
|     18|    588|     5|
|     19|    588|     4|
|     20|    588|     3|
|     21|    588|     4|
|     22|    588|     3|
|     23|    588|     3|
|     24|    588|     4|
|     25|    588|     5|
|     26|    588|     3|
|     27|    588|     4|
|     29|    588|     3|
|     33|    588|     3|
|     35|    588|     4|
|     37|    588|     3|
+-------+-------+------+
only showing top 20 rows

None


## Create ALS model

In [18]:
(train, test) = all_data.randomSplit([0.7, 0.3], seed=42)

In [19]:
# cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
# numBlocks (default 10): The number of blocks the users and items will be partitioned into in order to parallelize the computation.
# maxIter: Number of iterations.(default 10)
# regParam: Regularization parameter in ALS to prevent overfitting. The default value is 1.
# implicitPrefs: Boolean (True/False). Corresponds to if explicit or implicit feedback to use.
# explicit preferences given by the user to the item, for example.

# https://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html

als = ALS(maxIter=10, regParam=0.1, userCol="user_id", itemCol="game_id", ratingCol="rating", coldStartStrategy="drop", implicitPrefs=False)
model = als.fit(train)

In [20]:
# Evaluate the mode
predictions = model.transform(test)
predictions.show()

+-------+-------+------+--------------------+------------+--------------------+----------+----------+
|game_id|user_id|rating|                name|release_date|             summary|meta_score|prediction|
+-------+-------+------+--------------------+------------+--------------------+----------+----------+
|      1|    588|     5|The Legend of Zel...|   23-Nov-98|As a young boy, L...|        99| 3.7248714|
|      1|   2487|     4|The Legend of Zel...|   23-Nov-98|As a young boy, L...|        99| 3.5390494|
|      1|   3662|     4|The Legend of Zel...|   23-Nov-98|As a young boy, L...|        99|  4.901407|
|      1|   3922|     5|The Legend of Zel...|   23-Nov-98|As a young boy, L...|        99| 4.3520236|
|      1|   6630|     5|The Legend of Zel...|   23-Nov-98|As a young boy, L...|        99| 4.0468984|
|      1|   7563|     3|The Legend of Zel...|   23-Nov-98|As a young boy, L...|        99| 3.6409101|
|      1|   9246|     1|The Legend of Zel...|   23-Nov-98|As a young boy, L...|   

## Evaluate ALS

In [21]:
evaluator = RegressionEvaluator(metricName= "rmse", labelCol="rating", predictionCol="prediction")

In [22]:
rmse = evaluator.evaluate(predictions)
print(f"RMSE Error rate: {rmse}")

RMSE Error rate: 0.9232783575332598


## Build Recommender

In [23]:
def getusers (data, id):
  return data.filter(data['user_id'] == str(id)).select(["game_id", 'user_id'])

In [24]:
specific_user = getusers(test, 588 )
specific_user.show()

+-------+-------+
|game_id|user_id|
+-------+-------+
|      1|    588|
|      9|    588|
|     11|    588|
|     19|    588|
|     20|    588|
|     27|    588|
|     29|    588|
|     33|    588|
|     45|    588|
|     49|    588|
|     67|    588|
|     68|    588|
|     79|    588|
|     81|    588|
|    101|    588|
|    112|    588|
|    124|    588|
|    161|    588|
|    179|    588|
|    213|    588|
+-------+-------+
only showing top 20 rows



In [25]:
result = model.transform(specific_user)
result.orderBy("prediction", ascending = False).show()

+-------+-------+----------+
|game_id|user_id|prediction|
+-------+-------+----------+
|    161|    588| 4.2320056|
|     19|    588| 4.1328993|
|    516|    588| 4.1308217|
|    368|    588|  4.081168|
|    377|    588|  4.080956|
|    663|    588| 4.0223837|
|    669|    588|  3.986939|
|     27|    588| 3.9719193|
|     68|    588| 3.9556935|
|    446|    588|  3.952763|
|    534|    588| 3.9521356|
|    671|    588|  3.886736|
|    124|    588|  3.813913|
|     81|    588| 3.7848575|
|    538|    588| 3.7780554|
|    544|    588| 3.7766187|
|    213|    588|   3.77581|
|    179|    588| 3.7623775|
|    480|    588| 3.7386234|
|      1|    588| 3.7248714|
+-------+-------+----------+
only showing top 20 rows

